<a href="https://colab.research.google.com/github/Kathy42xu/DL_TA/blob/main/lstm_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 启用 GPU
import tensorflow as tf
device = tf.test.gpu_device_name()
print("GPU:", device)

# 安装依赖（若需要）
!pip install yfinance ta

# 下载数据 (示例：S&P500)
import yfinance as yf
df = yf.download("^GSPC", start="2002-08-01", end="2018-06-28")
df.to_csv("SP500.csv")


GPU: 
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6b419e1b8001f55d536572bec662d573b0679600c97d3c1690658058d529e76f
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


#preprocessing_indicator

In [ ]:
import pandas as pd

# 1️⃣ 重新读取 CSV —— 明确告诉 pandas 用第一列作 index
df = pd.read_csv("SP500.csv", index_col=0, parse_dates=True)

# 2️⃣ 确认列名正确，只保留 Open/High/Low/Close/Volume
print(df.columns)
df = df[['Open','High','Low','Close','Volume']]

# 3️⃣ 强制转成浮点数（会把任何非数字变成 NaN）
df = df.apply(pd.to_numeric, errors='coerce')

# 4️⃣ 删除因转换失败产生的 NaN 行
df.dropna(inplace=True)

print(df.head())


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
                  Open        High         Low       Close        Volume
Price                                                                   
2002-08-01  911.619995  911.619995  882.479980  884.659973  1.672200e+09
2002-08-02  884.400024  884.719971  853.950012  864.239990  1.538100e+09
2002-08-05  864.239990  864.239990  833.440002  834.599976  1.425500e+09
2002-08-06  834.599976  874.440002  834.599976  859.570007  1.514100e+09
2002-08-07  859.570007  878.739990  854.150024  876.770020  1.490400e+09


<ipython-input-3-dcf6b7282b58>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("SP500.csv", index_col=0, parse_dates=True)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ta

# -------------------------------
# 1) Read CSV with date parsing
# -------------------------------
df = pd.read_csv("SP500.csv", index_col=0, parse_dates=True)

# -------------------------------
# 2) Keep only OHLCV and convert to float
# -------------------------------
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df = df.apply(pd.to_numeric, errors='coerce')
df.dropna(inplace=True)

###############################################################################
# 3) Compute 43 Technical Indicators (Computed Features)
###############################################################################
# We'll compute 43 computed indicators, so that with the 5 raw columns,
# the final DataFrame has 43 + 5 = 48 columns.

# (1) Standard Deviation over 20 days (SD_20)
df['SD_20'] = df['Close'].rolling(20).std()

# (2-4) Simple Moving Averages (SMA)
df['SMA_5']  = ta.trend.sma_indicator(df['Close'], window=5)
df['SMA_10'] = ta.trend.sma_indicator(df['Close'], window=10)
df['SMA_20'] = ta.trend.sma_indicator(df['Close'], window=20)

# (5-7) Exponential Moving Averages (EMA)
df['EMA_6']  = ta.trend.ema_indicator(df['Close'], window=6)
df['EMA_10'] = ta.trend.ema_indicator(df['Close'], window=10)
df['EMA_14'] = ta.trend.ema_indicator(df['Close'], window=14)

# (8) MACD with fast=6, slow=12
df['MACD_6_12'] = ta.trend.macd_diff(df['Close'], window_slow=12, window_fast=6)

# (9-10) Relative Strength Index (RSI)
df['RSI_10'] = ta.momentum.rsi(df['Close'], window=10)
df['RSI_14'] = ta.momentum.rsi(df['Close'], window=14)

# (11) Commodity Channel Index (CCI)
df['CCI_20'] = ta.trend.cci(df['High'], df['Low'], df['Close'], window=20)

# (12-13) Bollinger Bands (Upper and Lower)
df['BOLL_UB'] = ta.volatility.bollinger_hband(df['Close'], window=20)
df['BOLL_LB'] = ta.volatility.bollinger_lband(df['Close'], window=20)

# (14) Average True Range (ATR)
df['ATR_14'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=14)

# (15-18) True Range Components: H-L, H-Cp, L-Cp, TR
df['H-L']  = df['High'] - df['Low']
df['H-Cp'] = (df['High'] - df['Close'].shift(1)).abs()
df['L-Cp'] = (df['Low']  - df['Close'].shift(1)).abs()
df['TR']   = df[['H-L', 'H-Cp', 'L-Cp']].max(axis=1)

# (19) On-Balance Volume (OBV)
df['OBV'] = ta.volume.on_balance_volume(df['Close'], df['Volume'])

# (20) Money Flow Index (MFI) using library
df['MFI'] = ta.volume.money_flow_index(df['High'], df['Low'], df['Close'], df['Volume'], window=14)

# (21) Force Index (1-day)
df['ForceIndex'] = ta.volume.force_index(df['Close'], df['Volume'], window=1)
# (22) 5-day Force Index (FI_5)
df['FI_5'] = ta.volume.force_index(df['Close'], df['Volume'], window=5)

# (23-24) Price and Volume Change Percentages (C%, V%)
df['C%'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
df['V%'] = (df['Volume'] - df['Volume'].shift(1)) / df['Volume'].shift(1)

# (25) Negative Volume Index (NVI)
df['NVI'] = ta.volume.negative_volume_index(df['Close'], df['Volume'])

# (26) Ease of Movement (SEMV)
df['SEMV'] = ta.volume.ease_of_movement(df['High'], df['Low'], df['Volume'])

# (27) True Strength Index (TSI) – custom calculation
def tsi(series, long=25, short=13):
    diff = series.diff()
    abs_diff = diff.abs()
    ema1 = diff.ewm(span=long, adjust=False).mean()
    ema2 = ema1.ewm(span=short, adjust=False).mean()
    abs_ema1 = abs_diff.ewm(span=long, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=short, adjust=False).mean()
    return 100 * ema2 / abs_ema2

df['TSI'] = tsi(df['Close'], long=25, short=13)

# (28) Money Flow Ratio (MFR) – computed manually
df['PMF'] = np.where(df['Close'] > df['Close'].shift(1), df['Close'] * df['Volume'], 0)
df['NMF'] = np.where(df['Close'] < df['Close'].shift(1), df['Close'] * df['Volume'], 0)
df['PMF_roll'] = df['PMF'].rolling(14).sum()
df['NMF_roll'] = df['NMF'].rolling(14).sum()
df['MFR'] = df['PMF_roll'] / df['NMF_roll']

# (29) We'll include the library MFI as an additional indicator;
#     This gives you two MFI-related features: MFI (from above) and a custom one.
df['MFI_custom'] = 100 - (100 / (1 + df['MFR']))

# (30-31) Vortex Indicators: Upper (UVI) and Lower (LVI)
df['UVI'] = ta.trend.vortex_indicator_pos(df['High'], df['Low'], df['Close'], window=14)
df['LVI'] = ta.trend.vortex_indicator_neg(df['High'], df['Low'], df['Close'], window=14)

# (32-33) Know Sure Thing (KST) and its 9-day SMA signal (KST_9)
df['KST'] = ta.trend.kst(df['Close'])
df['KST_9'] = df['KST'].rolling(window=9).mean()

# (34) Detrended Price Oscillator (DPO, 20-day)
df['DPO_20'] = ta.trend.dpo(df['Close'], window=20)

# (35) Directional Index (DX) computed from ADX+ and ADX-
adx_pos_14 = ta.trend.adx_pos(df['High'], df['Low'], df['Close'], window=14)
adx_neg_14 = ta.trend.adx_neg(df['High'], df['Low'], df['Close'], window=14)
df['DX'] = 100 * (adx_pos_14 - adx_neg_14).abs() / (adx_pos_14 + adx_neg_14)

# (36-37) Average Directional Index (ADX) for 7-day and 14-day
df['ADX_7'] = ta.trend.adx(df['High'], df['Low'], df['Close'], window=7)
df['ADX_14'] = ta.trend.adx(df['High'], df['Low'], df['Close'], window=14)

# (38) Rate of Change (ROC, 12-day)
df['ROC_12'] = ta.momentum.roc(df['Close'], window=12)

# (39) Williams %R (with lookback period 14)
df['Williams_%R'] = ta.momentum.williams_r(df['High'], df['Low'], df['Close'], lbp=14)

# --- Extra 5 to reach 43 computed indicators ---

# (40) EMA of Volume (10-day)
df['EMA_Volume'] = ta.trend.ema_indicator(df['Volume'], window=10)

# (41) Rate of Change (ROC, 5-day)
df['ROC_5'] = ta.momentum.roc(df['Close'], window=5)

# (42) Bollinger Bandwidth: (UB - LB) / SMA_20
df['BOLL_BW'] = (df['BOLL_UB'] - df['BOLL_LB']) / df['SMA_20']

# (43) ATR Ratio: ATR_14 divided by SMA_20
df['ATR_Ratio'] = df['ATR_14'] / df['SMA_20']

# (44) Volatility: Coefficient of Variation = SD_20 / SMA_20
df['Vol_CV'] = df['SD_20'] / df['SMA_20']

###############################################################################
# Now, decide which computed indicators to keep.
# We want exactly 43 computed indicators. Currently, after the raw OHLCV,
# our DataFrame has extra intermediate columns.
#
# The current columns list (after computation) is:
# ['Open', 'High', 'Low', 'Close', 'Volume', 'SD_20', 'SMA_5', 'SMA_10', 'SMA_20',
#  'EMA_6', 'EMA_10', 'EMA_14', 'MACD_6_12', 'RSI_10', 'RSI_14', 'CCI_20',
#  'BOLL_UB', 'BOLL_LB', 'ATR_14', 'H-L', 'H-Cp', 'L-Cp', 'TR', 'OBV', 'MFI',
#  'ForceIndex', 'FI_5', 'C%', 'V%', 'NVI', 'SEMV', 'TSI', 'PMF', 'NMF', 'PMF_roll',
#  'NMF_roll', 'MFR', 'MFI_custom', 'UVI', 'LVI', 'KST', 'KST_9', 'DPO_20', 'DX',
#  'ADX_7', 'ADX_14', 'ROC_12', 'Williams_%R', 'EMA_Volume', 'ROC_5', 'BOLL_BW',
#  'ATR_Ratio', 'Vol_CV']
#
# We need to select 43 computed features from these.
#
# One common choice (based on the paper's indicator list) is to keep:
#    SMA_5, SMA_10, SMA_20,
#    EMA_6, EMA_10, EMA_14,
#    MACD_6_12,
#    RSI_10, RSI_14,
#    CCI_20,
#    BOLL_UB, BOLL_LB,
#    ATR_14,
#    H-L, H-Cp, L-Cp, TR,
#    OBV,
#    MFI,            # (library version)
#    ForceIndex, FI_5,
#    C%, V%,
#    NVI,
#    SEMV,
#    TSI,
#    MFR,            # Money Flow Ratio
#    UVI, LVI,
#    KST, KST_9,
#    DPO_20,
#    DX,
#    ADX_7, ADX_14,
#    ROC_12,
#    Williams_%R,
#    EMA_Volume,     # extra
#    ROC_5,          # extra
#    BOLL_BW,        # extra
#    ATR_Ratio,      # extra
#    Vol_CV          # extra
#
# That is a total of 43 computed indicators.
###############################################################################

computed_features = ['SD_20','SMA_5', 'SMA_10', 'SMA_20',
                     'EMA_6', 'EMA_10', 'EMA_14',
                     'MACD_6_12',
                     'RSI_10', 'RSI_14',
                     'CCI_20',
                     'BOLL_UB', 'BOLL_LB',
                     'ATR_14',
                     'H-L', 'H-Cp', 'L-Cp', 'TR',
                     'OBV',
                     'MFI',
                     'ForceIndex', 'FI_5',
                     'C%', 'V%',
                     'NVI',
                     'SEMV',
                     'TSI',
                     'MFR',
                     'UVI', 'LVI',
                     'KST', 'KST_9',
                     'DPO_20',
                     'DX',
                     'ADX_7', 'ADX_14',
                     'ROC_12',
                     'Williams_%R',
                     'EMA_Volume',
                     'ROC_5',
                     'BOLL_BW',
                     'ATR_Ratio',
                     'Vol_CV']

# Now, the final DataFrame should contain the 5 raw columns + these 43 computed features.
final_cols = ['Open', 'High', 'Low', 'Close', 'Volume'] + computed_features
df_final = df[final_cols].copy()

###############################################################################
# 4) Normalization
###############################################################################
scaler = MinMaxScaler()
df_final[final_cols] = scaler.fit_transform(df_final[final_cols])

###############################################################################
# 5) Construct 50-day supervised sequences
###############################################################################
window = 50
# Compute next-day log returns (×100)
returns = 100 * np.log(df_final['Close'] / df_final['Close'].shift(1))
returns = returns.dropna()  # drop the first NaN row

X, y_reg, y_clf = [], [], []
for i in range(len(returns) - window):
    seq = df_final.iloc[i : i+window].values  # shape: (50, 48)
    X.append(seq)
    ret_val = returns.iloc[i+window]
    y_reg.append(ret_val)
    y_clf.append(int(ret_val > 0))

X = np.array(X)
y_reg = np.array(y_reg)
y_clf = np.array(y_clf)

###############################################################################
# 6) Save final dataset
###############################################################################
np.savez("SP500_preprocessed.npz", X=X, y_reg=y_reg, y_clf=y_clf)

print("Done!")
print("X shape:", X.shape)
print("y_reg shape:", y_reg.shape)
print("y_clf shape:", y_clf.shape)


<ipython-input-4-95c388c57979>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("SP500.csv", index_col=0, parse_dates=True)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Done!
X shape: (3954, 50, 48)
y_reg shape: (3954,)
y_clf shape: (3954,)


#Train LSTM

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

data = np.load("SP500_preprocessed.npz")
X, y_reg, y_clf = data['X'], data['y_reg'], data['y_clf']

# 按 70% train / 10% validation / 20% test 切分
X_temp, X_test, y_reg_temp, y_reg_test, y_clf_temp, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, shuffle=False)

X_train, X_val, y_reg_train, y_reg_val, y_clf_train, y_clf_val = train_test_split(
    X_temp, y_reg_temp, y_clf_temp, test_size=0.125, shuffle=False)

print("Shapes → Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Shapes → Train: (2767, 50, 48) Val: (396, 50, 48) Test: (791, 50, 48)


In [ ]:
mask = (~np.isnan(y_reg)) & (~np.isinf(y_reg))
X, y_reg, y_clf = X[mask], y_reg[mask], y_clf[mask]


##LFM

In [ ]:
# 1️⃣ Split data exactly as authors do
from sklearn.model_selection import train_test_split
X_temp, X_test, y_reg_temp, y_reg_test, y_clf_temp, y_clf_test = train_test_split(X, y_reg, y_clf, test_size=0.2, shuffle=False)
X_train, X_val, y_reg_train, y_reg_val, y_clf_train, y_clf_val = train_test_split(X_temp, y_reg_temp, y_clf_temp, test_size=0.125, shuffle=False)

# 2️⃣ Build exactly as original script
def build_model():
    inp = Input(shape=(50, X.shape[2]))
    x = LSTM(15, return_sequences=False)(inp)
    x = Dense(30, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(30, activation="relu")(x)
    out_reg = Dense(1, name="regression")(x)
    out_clf = Dense(2, activation="softmax", name="classification")(x)
    model = Model(inp, [out_reg, out_clf])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss={"regression":"mse", "classification":"sparse_categorical_crossentropy"},
        loss_weights={"regression":0.1, "classification":1.0}
    )
    return model

model = build_model()

# 3️⃣ Validate no NaNs in training batches
print("Train X NaNs:", np.isnan(X_train).sum(), "y_reg NaNs:", np.isnan(y_reg_train).sum())

# 4️⃣ Train
history = model.fit(
    X_train, {"regression":y_reg_train, "classification":y_clf_train},
    validation_data=(X_val, {"regression":y_reg_val, "classification":y_clf_val}),
    batch_size=256, epochs=300, callbacks=[EarlyStopping(patience=20, restore_best_weights=True)]
)


Train X NaNs: 0 y_reg NaNs: 0
Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 92ms/step - classification_loss: 0.6887 - loss: 5.0865 - regression_loss: 43.8788 - val_classification_loss: 0.6907 - val_loss: 0.8137 - val_regression_loss: 1.2653
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - classification_loss: 0.6914 - loss: 5.1222 - regression_loss: 44.2889 - val_classification_loss: 0.6910 - val_loss: 0.8129 - val_regression_loss: 1.2580
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - classification_loss: 0.6890 - loss: 4.1000 - regression_loss: 34.0164 - val_classification_loss: 0.6914 - val_loss: 0.8134 - val_regression_loss: 1.2609
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - classification_loss: 0.6871 - loss: 3.7978 - regression_loss: 30.9953 - val_classification_loss: 0.6912 - val_loss: 0.8128 - val_regression_loss: 1.2579
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - classification_loss: 0.6905 - loss: 3.1821 - regression_loss: 25.5191 - val_classificati

## paper logic:100 LSTM base model + learning rate halved every 50 epochs + 300 epochs fixed training + 13 randomly drawn features + bootstrap samples + final average prediction


In [ ]:
import numpy as np
import random
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import clone_model

def build_model(input_dim):
    inp = Input(shape=(50, input_dim))
    x = LSTM(15)(inp)
    x = Dense(30, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(30, activation="relu")(x)
    out_reg = Dense(1, name="regression")(x)
    out_clf = Dense(2, activation="softmax", name="classification")(x)
    model = Model(inp, [out_reg, out_clf])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss={"regression":"mse", "classification":"sparse_categorical_crossentropy"},
        loss_weights={"regression":0.1, "classification":1.0}
    )
    return model





# Learning rate schedule: halve every 50 epochs
def lr_schedule(epoch):
    return 1e-3 * (0.5 ** (epoch // 50))

lr_cb = LearningRateScheduler(lr_schedule)

ensemble_preds_reg = []
ensemble_preds_clf = []

for i in range(100):
    # Randomly sample 13 features
    features_idx = random.sample(range(X_train.shape[2]), 13)
    X_tr_sub = X_train[:,:,features_idx]
    X_val_sub = X_val[:,:,features_idx]
    X_test_sub = X_test[:,:,features_idx]

    # Bootstrap sample training data
    idx = np.random.choice(len(X_tr_sub), size=len(X_tr_sub), replace=True)
    X_boot, y_reg_boot, y_clf_boot = X_tr_sub[idx], y_reg_train[idx], y_clf_train[idx]

    # Build & train
    model_i = build_model(input_dim=X_train.shape[2] if False else X_boot.shape[2])
    model_i.fit(
        X_boot, {"regression": y_reg_boot, "classification": y_clf_boot},
        validation_data=(X_val_sub, {"regression": y_reg_val, "classification": y_clf_val}),
        epochs=300, batch_size=256, callbacks=[lr_cb], verbose=0
    )

    # Predict on test
    reg_pred, clf_pred = model_i.predict(X_test_sub, verbose=0)
    ensemble_preds_reg.append(reg_pred.flatten())
    ensemble_preds_clf.append(clf_pred)

# Average ensemble outputs
final_reg = np.mean(np.vstack(ensemble_preds_reg), axis=0)
final_clf = np.mean(np.stack(ensemble_preds_clf), axis=0).argmax(axis=1)

print("Ensemble Test RMSE:", np.sqrt(((final_reg - y_reg_test)**2).mean()))
from sklearn.metrics import balanced_accuracy_score
print("Ensemble Test BACC:", balanced_accuracy_score(y_clf_test, final_clf))


#multi task


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# --- Assume df_final has been created as in your preprocessing cell ---
# final_cols = ['Open','High','Low','Close','Volume'] + computed_features
# df_final = df[final_cols].copy()

# Check if any NaNs remain in df_final
print("NaNs per column in final DataFrame:")
print(df_final.isna().sum())

# Normalize the final DataFrame
scaler = MinMaxScaler()
final_cols = df_final.columns.tolist()
df_final[final_cols] = scaler.fit_transform(df_final[final_cols])

# Compute next-day log returns and drop the first row
returns = 100 * np.log(df_final['Close'] / df_final['Close'].shift(1))
returns = returns.dropna()

# Construct 50-day sequences
window = 50
X, y_reg, y_clf = [], [], []
for i in range(len(returns) - window):
    seq = df_final.iloc[i : i+window].values  # each seq: shape (50, 48)
    X.append(seq)
    ret_val = returns.iloc[i+window]
    y_reg.append(ret_val)
    y_clf.append(int(ret_val > 0))

X = np.array(X)
y_reg = np.array(y_reg)
y_clf = np.array(y_clf)

print("Before filtering, X shape:", X.shape)

# Filter out any sequences that contain NaN values
valid_mask = ~np.isnan(X).any(axis=(1, 2))
X = X[valid_mask]
y_reg = y_reg[valid_mask]
y_clf = y_clf[valid_mask]

print("After filtering, X shape:", X.shape)
print("y_reg shape:", y_reg.shape, "y_clf shape:", y_clf.shape)

# Save the dataset
np.savez("SP500_preprocessed.npz", X=X, y_reg=y_reg, y_clf=y_clf)
print("Dataset saved successfully!")


NaNs per column in final DataFrame:
Open            0
High            0
Low             0
Close           0
Volume          0
SD_20          19
SMA_5           4
SMA_10          9
SMA_20         19
EMA_6           5
EMA_10          9
EMA_14         13
MACD_6_12      19
RSI_10          9
RSI_14         13
CCI_20         19
BOLL_UB        19
BOLL_LB        19
ATR_14          0
H-L             0
H-Cp            1
L-Cp            1
TR              0
OBV             0
MFI            13
ForceIndex      1
FI_5            5
C%              1
V%              1
NVI             0
SEMV            1
TSI             1
MFR            13
UVI            14
LVI            14
KST            14
KST_9          22
DPO_20         19
DX             15
ADX_7           0
ADX_14          0
ROC_12         12
Williams_%R    13
EMA_Volume      9
ROC_5           5
BOLL_BW        19
ATR_Ratio      19
Vol_CV         19
dtype: int64


/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Before filtering, X shape: (3954, 50, 48)
After filtering, X shape: (3932, 50, 48)
y_reg shape: (3932,) y_clf shape: (3932,)
Dataset saved successfully!


In [ ]:
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, balanced_accuracy_score

# ------------------------------
# 1) Load the preprocessed dataset
# ------------------------------
data = np.load("SP500_preprocessed.npz")
X, y_reg, y_clf = data['X'], data['y_reg'], data['y_clf']

# ------------------------------
# 2) Data Cleaning (as in one-task version)
# ------------------------------
# Remove any samples with NaN or Inf in regression targets
mask = (~np.isnan(y_reg)) & (~np.isinf(y_reg))
X, y_reg, y_clf = X[mask], y_reg[mask], y_clf[mask]
# Optionally clip extreme regression targets to avoid instability
y_reg = np.clip(y_reg, -10, 10)

# ------------------------------
# 3) Split data exactly as authors do:
#    70% train / 10% validation / 20% test (no shuffling)
# ------------------------------
X_temp, X_test, y_reg_temp, y_reg_test, y_clf_temp, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, shuffle=False)
X_train, X_val, y_reg_train, y_reg_val, y_clf_train, y_clf_val = train_test_split(
    X_temp, y_reg_temp, y_clf_temp, test_size=0.125, shuffle=False)

print("Shapes → Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# ------------------------------
# 4) Build the multitask LSTM model exactly as in your one-task version
# ------------------------------
def build_model():
    inp = Input(shape=(50, X.shape[2]))  # X.shape[2] should be 48 (5 raw + 43 computed)
    x = LSTM(15, return_sequences=False)(inp)
    x = Dense(30, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(30, activation="relu")(x)
    out_reg = Dense(1, name="regression")(x)
    out_clf = Dense(2, activation="softmax", name="classification")(x)
    model = Model(inputs=inp, outputs=[out_reg, out_clf])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss={"regression": "mse", "classification": "sparse_categorical_crossentropy"},
                  loss_weights={"regression": 0.1, "classification": 1.0})
    return model

model = build_model()
model.summary()

# Verify that training data has no NaNs
print("Train X NaNs:", np.isnan(X_train).sum(), "y_reg NaNs:", np.isnan(y_reg_train).sum())

# ------------------------------
# 5) Define learning rate schedule and early stopping
# ------------------------------
def lr_schedule(epoch, lr):
    # Halve the learning rate every 50 epochs
    return 1e-3 * (0.5 ** (epoch // 50))

lr_cb = LearningRateScheduler(lr_schedule)
es_cb = EarlyStopping(patience=20, restore_best_weights=True)

# ------------------------------
# 6) Train the model
# ------------------------------
history = model.fit(
    X_train, {"regression": y_reg_train, "classification": y_clf_train},
    validation_data=(X_val, {"regression": y_reg_val, "classification": y_clf_val}),
    epochs=300,
    batch_size=256,
    callbacks=[lr_cb, es_cb],
    verbose=1
)

# ------------------------------
# 7) Evaluate the model on the test set
# ------------------------------
pred_reg, pred_clf = model.predict(X_test, verbose=0)
pred_clf = pred_clf.argmax(axis=1)

rmse = math.sqrt(mean_squared_error(y_reg_test, pred_reg))
acc = accuracy_score(y_clf_test, pred_clf)
bacc = balanced_accuracy_score(y_clf_test, pred_clf)

print("Test RMSE:", rmse)
print("Test Accuracy:", acc)
print("Test Balanced Accuracy:", bacc)


Shapes → Train: (2751, 50, 48) Val: (393, 50, 48) Test: (786, 50, 48)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 50, 48)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_4 (LSTM)             │ (None, 15)             │          3,840 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 30)             │            480 │ lstm_4[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 30)             │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 30)             │            930 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ regression (Dense)        │ (None, 1)              │             31 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification (Dense)    │ (None, 2)              │             62 │ dense_9[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 5,343 (20.87 KB)

 Trainable params: 5,343 (20.87 KB)

 Non-trainable params: 0 (0.00 B)

Train X NaNs: 0 y_reg NaNs: 0
Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 90ms/step - classification_loss: 0.6903 - loss: 1.6974 - regression_loss: 10.0700 - val_classification_loss: 0.6921 - val_loss: 0.8132 - val_regression_loss: 1.2572 - learning_rate: 0.0010
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - classification_loss: 0.6896 - loss: 1.6389 - regression_loss: 9.4846 - val_classification_loss: 0.6918 - val_loss: 0.8133 - val_regression_loss: 1.2583 - learning_rate: 0.0010
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - classification_loss: 0.6904 - loss: 1.6894 - regression_loss: 9.9900 - val_classification_loss: 0.6919 - val_loss: 0.8132 - val_regression_loss: 1.2566 - learning_rate: 0.0010
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - classification_loss: 0.6905 - loss: 1.6520 - regression_loss: 9.6083 - val_classification_loss: 0.6922 - val_loss: 0.8131 - val_regression_loss: 1.2556 - learning_rate: 0.0010
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/s

In [ ]:
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, balanced_accuracy_score

# --------------------------------------------------
# 1) Load preprocessed dataset
# --------------------------------------------------
data = np.load("SP500_preprocessed.npz")
X, y_reg, y_clf = data['X'], data['y_reg'], data['y_clf']

# --------------------------------------------------
# 2) Clean the data (same as one-task version)
# --------------------------------------------------
mask = (~np.isnan(y_reg)) & (~np.isinf(y_reg))
X, y_reg, y_clf = X[mask], y_reg[mask], y_clf[mask]
# Clip regression targets to a reasonable range (e.g., [-10, 10])
y_reg = np.clip(y_reg, -10, 10)

# --------------------------------------------------
# 3) Split data exactly as authors do:
#    70% train / 10% validation / 20% test (without shuffling)
# --------------------------------------------------
X_temp, X_test, y_reg_temp, y_reg_test, y_clf_temp, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, shuffle=False)
X_train, X_val, y_reg_train, y_reg_val, y_clf_train, y_clf_val = train_test_split(
    X_temp, y_reg_temp, y_clf_temp, test_size=0.125, shuffle=False)

print("Shapes → Train:", X_train.shape, "Validation:", X_val.shape, "Test:", X_test.shape)

# --------------------------------------------------
# 4) Build the multitask model (exact paper setting)
# --------------------------------------------------
def build_multitask_model(input_shape):
    inp = Input(shape=input_shape)  # Expected input shape: (50, 48)

    # Shared layers:
    x = LSTM(15, return_sequences=False)(inp)
    x = Dense(30, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(30, activation='relu')(x)

    # Regression task-specific branch (3 FC layers of 30 neurons each)
    reg_branch = Dense(30, activation='relu')(x)
    reg_branch = Dense(30, activation='relu')(reg_branch)
    reg_branch = Dense(30, activation='relu')(reg_branch)
    out_reg = Dense(1, name='regression')(reg_branch)

    # Classification task-specific branch (3 FC layers of 30 neurons each)
    clf_branch = Dense(30, activation='relu')(x)
    clf_branch = Dense(30, activation='relu')(clf_branch)
    clf_branch = Dense(30, activation='relu')(clf_branch)
    out_clf = Dense(2, activation='softmax', name='classification')(clf_branch)

    model = Model(inputs=inp, outputs=[out_reg, out_clf])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss={'regression': 'mse', 'classification': 'sparse_categorical_crossentropy'},
                  loss_weights={'regression': 0.1, 'classification': 1.0})
    return model

model = build_multitask_model((X_train.shape[1], X_train.shape[2]))
model.summary()

# --------------------------------------------------
# 5) Define learning rate scheduler and early stopping
# --------------------------------------------------
def lr_schedule(epoch, lr):
    # Halve the learning rate every 50 epochs
    return 1e-3 * (0.5 ** (epoch // 50))

lr_cb = LearningRateScheduler(lr_schedule)
es_cb = EarlyStopping(patience=20, restore_best_weights=True)

# --------------------------------------------------
# 6) Train the model for 300 epochs
# --------------------------------------------------
history = model.fit(
    X_train, {'regression': y_reg_train, 'classification': y_clf_train},
    validation_data=(X_val, {'regression': y_reg_val, 'classification': y_clf_val}),
    epochs=300,
    batch_size=256,
    callbacks=[lr_cb, es_cb],
    verbose=1
)

# --------------------------------------------------
# 7) Evaluate the model on the test set
# --------------------------------------------------
pred_reg, pred_clf = model.predict(X_test, verbose=0)
pred_clf = pred_clf.argmax(axis=1)

rmse = math.sqrt(mean_squared_error(y_reg_test, pred_reg))
acc = accuracy_score(y_clf_test, pred_clf)
bacc = balanced_accuracy_score(y_clf_test, pred_clf)

print("Test RMSE:", rmse)
print("Test Accuracy:", acc)
print("Test Balanced Accuracy:", bacc)


Shapes → Train: (2751, 50, 48) Validation: (393, 50, 48) Test: (786, 50, 48)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 50, 48)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 15)             │          3,840 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 30)             │            480 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 30)             │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 30)             │            930 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 30)             │            930 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 30)             │            930 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 30)             │            930 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 30)             │            930 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 30)             │            930 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 30)             │            930 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ regression (Dense)        │ (None, 1)              │             31 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification (Dense)    │ (None, 2)              │             62 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,923 (42.67 KB)

 Trainable params: 10,923 (42.67 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 93ms/step - classification_loss: 0.6910 - loss: 1.7049 - regression_loss: 10.1348 - val_classification_loss: 0.6932 - val_loss: 0.8142 - val_regression_loss: 1.2630 - learning_rate: 0.0010
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - classification_loss: 0.6880 - loss: 1.6324 - regression_loss: 9.4481 - val_classification_loss: 0.6920 - val_loss: 0.8128 - val_regression_loss: 1.2572 - learning_rate: 0.0010
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - classification_loss: 0.6861 - loss: 1.6804 - regression_loss: 9.9304 - val_classification_loss: 0.6909 - val_loss: 0.8120 - val_regression_loss: 1.2571 - learning_rate: 0.0010
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - classification_loss: 0.6885 - loss: 1.6613 - regression_loss: 9.7289 - val_classification_loss: 0.6907 - val_loss: 0.8119 - val_regression_loss: 1.2568 - learning_rate: 0.0010
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - classification_loss: 0.68

In [ ]:
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, balanced_accuracy_score

# ------------------------------
# 1) Load preprocessed dataset
# ------------------------------
data = np.load("SP500_preprocessed.npz")
X, y_reg, y_clf = data['X'], data['y_reg'], data['y_clf']

# ------------------------------
# 2) Clean the data (as in one-task version)
# ------------------------------
mask = (~np.isnan(y_reg)) & (~np.isinf(y_reg))
X, y_reg, y_clf = X[mask], y_reg[mask], y_clf[mask]
y_reg = np.clip(y_reg, -10, 10)  # Clip extreme values

# ------------------------------
# 3) Split data exactly as authors do:
#    70% train / 10% validation / 20% test (without shuffling)
# ------------------------------
X_temp, X_test, y_reg_temp, y_reg_test, y_clf_temp, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, shuffle=False)
X_train, X_val, y_reg_train, y_reg_val, y_clf_train, y_clf_val = train_test_split(
    X_temp, y_reg_temp, y_clf_temp, test_size=0.125, shuffle=False)

print("Shapes → Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# ------------------------------
# 4) Build the multitask model (same architecture as before)
# ------------------------------
def build_multitask_model():
    inp = Input(shape=(50, X.shape[2]))  # Expected input shape: (50, 48) if you have 48 features
    x = LSTM(15, return_sequences=False)(inp)
    x = Dense(30, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(30, activation='relu')(x)

    # Regression branch: 3 FC layers
    reg_branch = Dense(30, activation='relu')(x)
    reg_branch = Dense(30, activation='relu')(reg_branch)
    reg_branch = Dense(30, activation='relu')(reg_branch)
    out_reg = Dense(1, name='regression')(reg_branch)

    # Classification branch: 3 FC layers
    clf_branch = Dense(30, activation='relu')(x)
    clf_branch = Dense(30, activation='relu')(clf_branch)
    clf_branch = Dense(30, activation='relu')(clf_branch)
    out_clf = Dense(2, activation='softmax', name='classification')(clf_branch)

    model = Model(inputs=inp, outputs=[out_reg, out_clf])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss={'regression': 'mse', 'classification': 'sparse_categorical_crossentropy'},
                  loss_weights={'regression': 0.1, 'classification': 1.0})
    return model

model = build_multitask_model()
model.summary()

# Verify no NaNs in training data
print("Train X NaNs:", np.isnan(X_train).sum(), "y_reg NaNs:", np.isnan(y_reg_train).sum())

# ------------------------------
# 5) Set up learning rate scheduler and early stopping
# ------------------------------
def lr_schedule(epoch, lr):
    # Halve the learning rate every 50 epochs
    return 1e-3 * (0.5 ** (epoch // 50))

lr_cb = LearningRateScheduler(lr_schedule)
# Increase patience to 50 epochs for a longer training period
es_cb = EarlyStopping(patience=50, restore_best_weights=True)

# ------------------------------
# 6) Train the model for up to 500 epochs
# ------------------------------
history = model.fit(
    X_train, {'regression': y_reg_train, 'classification': y_clf_train},
    validation_data=(X_val, {'regression': y_reg_val, 'classification': y_clf_val}),
    epochs=500,
    batch_size=256,
    callbacks=[lr_cb, es_cb],
    verbose=1
)

# ------------------------------
# 7) Evaluate the model on the test set
# ------------------------------
pred_reg, pred_clf = model.predict(X_test, verbose=0)
pred_clf = pred_clf.argmax(axis=1)

rmse = math.sqrt(mean_squared_error(y_reg_test, pred_reg))
acc = accuracy_score(y_clf_test, pred_clf)
bacc = balanced_accuracy_score(y_clf_test, pred_clf)

print("Test RMSE:", rmse)
print("Test Accuracy:", acc)
print("Test Balanced Accuracy:", bacc)


Shapes → Train: (2751, 50, 48) Val: (393, 50, 48) Test: (786, 50, 48)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 50, 48)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 15)             │          3,840 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 30)             │            480 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 30)             │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 30)             │            930 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 30)             │            930 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 30)             │            930 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 30)             │            930 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 30)             │            930 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 30)             │            930 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 30)             │            930 │ dense_14[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ regression (Dense)        │ (None, 1)              │             31 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ classification (Dense)    │ (None, 2)              │             62 │ dense_15[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,923 (42.67 KB)

 Trainable params: 10,923 (42.67 KB)

 Non-trainable params: 0 (0.00 B)

Train X NaNs: 0 y_reg NaNs: 0
Epoch 1/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 87ms/step - classification_loss: 0.6926 - loss: 1.6778 - regression_loss: 9.8530 - val_classification_loss: 0.6911 - val_loss: 0.8130 - val_regression_loss: 1.2604 - learning_rate: 0.0010
Epoch 2/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - classification_loss: 0.6892 - loss: 1.6927 - regression_loss: 10.0295 - val_classification_loss: 0.6923 - val_loss: 0.8136 - val_regression_loss: 1.2609 - learning_rate: 0.0010
Epoch 3/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - classification_loss: 0.6878 - loss: 1.6393 - regression_loss: 9.5107 - val_classification_loss: 0.6924 - val_loss: 0.8136 - val_regression_loss: 1.2606 - learning_rate: 0.0010
Epoch 4/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - classification_loss: 0.6887 - loss: 1.6775 - regression_loss: 9.8884 - val_classification_loss: 0.6915 - val_loss: 0.8129 - val_regression_loss: 1.2597 - learning_rate: 0.0010
Epoch 5/500
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/s

In [ ]:
import numpy as np
import math
import random
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.metrics import mean_squared_error, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split

# ------------------------------
# 1) Load preprocessed dataset (48 features per timestep)
# ------------------------------
data = np.load("SP500_preprocessed.npz")
X, y_reg, y_clf = data['X'], data['y_reg'], data['y_clf']

# ------------------------------
# 2) Data cleaning (same as one-task)
# ------------------------------
mask = (~np.isnan(y_reg)) & (~np.isinf(y_reg))
X, y_reg, y_clf = X[mask], y_reg[mask], y_clf[mask]
y_reg = np.clip(y_reg, -10, 10)  # clip extreme targets

# ------------------------------
# 3) Split data exactly as authors do:
#     70% train / 10% validation / 20% test (without shuffling)
# ------------------------------
X_temp, X_test, y_reg_temp, y_reg_test, y_clf_temp, y_clf_test = train_test_split(
    X, y_reg, y_clf, test_size=0.2, shuffle=False)
X_train, X_val, y_reg_train, y_reg_val, y_clf_train, y_clf_val = train_test_split(
    X_temp, y_reg_temp, y_clf_temp, test_size=0.125, shuffle=False)

print("Shapes → Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# ------------------------------
# 4) Model building function (as per original paper)
# ------------------------------
def build_model(input_dim):
    inp = Input(shape=(50, input_dim))
    x = LSTM(15, return_sequences=False)(inp)
    x = Dense(30, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(30, activation="relu")(x)
    # Regression branch (3 Dense layers)
    reg_branch = Dense(30, activation="relu")(x)
    reg_branch = Dense(30, activation="relu")(reg_branch)
    reg_branch = Dense(30, activation="relu")(reg_branch)
    out_reg = Dense(1, name="regression")(reg_branch)
    # Classification branch (3 Dense layers)
    clf_branch = Dense(30, activation="relu")(x)
    clf_branch = Dense(30, activation="relu")(clf_branch)
    clf_branch = Dense(30, activation="relu")(clf_branch)
    out_clf = Dense(2, activation="softmax", name="classification")(clf_branch)

    model = Model(inputs=inp, outputs=[out_reg, out_clf])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss={"regression": "mse", "classification": "sparse_categorical_crossentropy"},
                  loss_weights={"regression": 0.1, "classification": 1.0})
    return model

# ------------------------------
# 5) Learning rate scheduler and early stopping
# ------------------------------
def lr_schedule(epoch, lr):
    # Halve the learning rate every 50 epochs
    return 1e-3 * (0.5 ** (epoch // 50))
lr_cb = LearningRateScheduler(lr_schedule)

# ------------------------------
# 6) Ensemble training settings
# ------------------------------
num_models = 100   # Number of ensemble models
feature_subset = 13  # Number of features to sample for each model

ensemble_preds_reg = []
ensemble_preds_clf = []

for i in range(num_models):
    print(f"Training model {i+1}/{num_models}")
    # Randomly sample feature indices from 0 to X_train.shape[2]-1
    feature_idx = sorted(random.sample(range(X_train.shape[2]), feature_subset))

    # Select features from training, validation, and test sets
    X_train_sub = X_train[:, :, feature_idx]
    X_val_sub   = X_val[:, :, feature_idx]
    X_test_sub  = X_test[:, :, feature_idx]

    # Bootstrap sample from training data (with replacement)
    indices = np.random.choice(len(X_train_sub), size=len(X_train_sub), replace=True)
    X_boot = X_train_sub[indices]
    y_reg_boot = y_reg_train[indices]
    y_clf_boot = y_clf_train[indices]

    # Build and train the model on this bootstrap sample
    model_i = build_model(input_dim=feature_subset)
    es_cb = EarlyStopping(patience=20, restore_best_weights=True)

    model_i.fit(
        X_boot, {"regression": y_reg_boot, "classification": y_clf_boot},
        validation_data=(X_val_sub, {"regression": y_reg_val, "classification": y_clf_val}),
        epochs=300, batch_size=256, callbacks=[lr_cb, es_cb], verbose=0
    )

    # Predict on test set using the same subset of features
    pred_reg_i, pred_clf_i = model_i.predict(X_test_sub, verbose=0)
    ensemble_preds_reg.append(pred_reg_i.flatten())
    ensemble_preds_clf.append(pred_clf_i)

# ------------------------------
# 7) Aggregate Ensemble Predictions
# ------------------------------
final_pred_reg = np.mean(np.vstack(ensemble_preds_reg), axis=0)
final_pred_clf_prob = np.mean(np.stack(ensemble_preds_clf), axis=0)
final_pred_clf = final_pred_clf_prob.argmax(axis=1)

# ------------------------------
# 8) Evaluate Ensemble Performance
# ------------------------------
rmse = math.sqrt(mean_squared_error(y_reg_test, final_pred_reg))
acc = accuracy_score(y_clf_test, final_pred_clf)
bacc = balanced_accuracy_score(y_clf_test, final_pred_clf)

print("Ensemble Test RMSE:", rmse)
print("Ensemble Test Accuracy:", acc)
print("Ensemble Test Balanced Accuracy:", bacc)


Shapes → Train: (2751, 50, 48) Val: (393, 50, 48) Test: (786, 50, 48)
Training model 1/100
Training model 2/100
Training model 3/100
Training model 4/100
Training model 5/100
Training model 6/100
Training model 7/100
Training model 8/100
Training model 9/100
Training model 10/100
Training model 11/100
Training model 12/100
Training model 13/100
Training model 14/100
Training model 15/100
Training model 16/100
Training model 17/100
Training model 18/100
Training model 19/100
Training model 20/100
Training model 21/100
Training model 22/100
Training model 23/100
Training model 24/100
Training model 25/100
Training model 26/100
Training model 27/100
Training model 28/100
Training model 29/100
Training model 30/100
Training model 31/100
Training model 32/100
Training model 33/100
Training model 34/100
Training model 35/100
Training model 36/100
Training model 37/100
Training model 38/100
Training model 39/100
Training model 40/100
Training model 41/100
Training model 42/100
Training model 